In [ ]:
import streamlit as st
import pandas as pd
import requests
import json

# Import Snowpark context for the active session
from snowflake.snowpark.context import get_active_session
session = get_active_session()

# **Query Snowflake Cortex Analyst API via Slack**

This notebook demonstrates how to integrate **Snowflake Cortex Analyst API** for real-time data analysis via **Slack**. Users can ask **natural language questions** in Slack, and Snowflake will use **Cortex Analyst** to generate SQL queries and provide instant answers.

### **End Result**

1. **Slack**: You ask a question, such as **"What's the average shipping time from Snowtires Automotive?"**
2. **Snowflake**: Using **Cortex Analyst API**, Snowflake generates the corresponding SQL and returns the result.

### **Step 1: Ask a Question via Slack**
Ask a question like **`/askcortex What's the average shipping time from Snowtires Automotive?`**. This triggers Snowflake to generate the SQL and provide the answer.

Example Output:\
**Answer:**\
AVERAGE_SHIPPING_TIME\
5.200000

`SELECT DISTINCT average_shipping_time FROM skicar.skicar_schema.supply_chain WHERE supplier_vendor_name = 'Snowtires Automotive' UNION SELECT 'Other Suppliers' AS supplier_vendor_name, AVG(average_shipping_time) FROM (SELECT DISTINCT (supplier_vendor_name), average_shipping_time FROM skicar.skicar_schema.supply_chain WHERE supplier_vendor_name <> 'Snowtires Automotive')
 -- Generated by Cortex Analyst
;`

## Snowflake Setup

The core components of the demo in Snowflake include:

- **Cortex Service Role**: This role allows us to interact with the Cortex API.
- **Warehouse**: Powers the queries executed by the Analyst.
- **Schemas and Tables**: We’ve set up the required schemas and tables, such as `SUPPLY_CHAIN` and `PRODUCT_INFO` for querying order details.

Here’s the SQL used to set it up:

In [ ]:
-- Create role and grant permissions
CREATE ROLE IF NOT EXISTS CORTEX_SERVICE_ROLE;
GRANT ROLE CORTEX_SERVICE_ROLE TO USER D4B;

-- Setup the Warehouse
CREATE OR REPLACE WAREHOUSE SKICAR WAREHOUSE_SIZE = XSMALL AUTO_SUSPEND = 60;
USE WAREHOUSE SKICAR;

-- Grant access to the warehouse
GRANT USAGE ON WAREHOUSE SKICAR TO ROLE CORTEX_SERVICE_ROLE;

## Data Ingestion

For this demo, we access data from two sources:

1. **SharePoint**: Non-streaming data such as product details and orders via **Snowflake Marketplace**

In [ ]:
# Define image in a stage and read the file
image=session.file.get_stream('@SKICAR.SKICAR_SCHEMA.SETUP/SharePoint.jpg' , decompress=False).read() 

# Display the image
st.image(image, width=1200)

In [ ]:
# Define image in a stage and read the file
image=session.file.get_stream('@SKICAR.SKICAR_SCHEMA.SETUP/SFMarketplace.jpg' , decompress=False).read() 

# Display the image
st.image(image, width=1200)

In [ ]:
-- Example of a table creation and data load from SharePoint
CREATE TABLE PRODUCT_INFO (
    PRODUCT_ID VARCHAR,
    PRODUCT_NAME VARCHAR,
    PRODUCT_TYPE VARCHAR
);

-- Insert data into PRODUCT_INFO
INSERT INTO PRODUCT_INFO VALUES
('SKU-123', 'All-Season Tires', 'TIRE');

2. **S3 Iceberg Tables**: Real-time streaming data stored in **Iceberg** format, directly queryable in Snowflake.

In [ ]:
# Define image in a stage and read the file
image=session.file.get_stream('@SKICAR.SKICAR_SCHEMA.SETUP/s3.jpg' , decompress=False).read() 

# Display the image
st.image(image, width=1200)

## **Querying Iceberg Tables in Snowflake**

You can **query Iceberg tables** in Snowflake directly. Iceberg is a **table format** supported by Snowflake, which allows you to perform queries using standard SQL.

### **Key Points**:
- **Standard SQL**: Once the data is stored in **Iceberg format**, you can query the table just like any other Snowflake table.

### **Example SQL Query**:
The following query retrieves records from an **Iceberg table**:

```sql
SELECT * 
FROM my_iceberg_table
WHERE order_status = 'IN_TRANSIT'
ORDER BY order_timestamp DESC;

## Data Pipeline Overview

This diagram shows the **data pipeline** for processing and visualizing data from two sources:

1. **SharePoint via Snowflake Marketplace Connector**: Data is ingested from SharePoint for further processing.
2. **S3 Iceberg Tables**: Real-time streaming data is accessed directly from Iceberg tables for querying.

### Key Points:
- **Dynamic Tables**: Used to process and apply incremental updates to the data in real-time.
- The DAG illustrates how data flows between the different tables, with dependencies showing the order of processing.

Below is the **graph** showing the flow of data through the pipeline.


In [ ]:
# Define image in a stage and read the file
image=session.file.get_stream('@SKICAR.SKICAR_SCHEMA.SETUP/DAG.jpg' , decompress=False).read() 

# Display the image
st.image(image, width=1200)

## **Semantic Model Overview**

The **Semantic Model** defines the business logic and structure for querying data. It maps raw data to meaningful terms, allowing natural language queries to be processed effectively.

### **Key Points**:
- **Defines Business Logic**: The Semantic Model translates raw data into understandable business terms.
- **Enables Natural Language Queries**: Allows **Cortex Analyst** to interpret user queries in natural language.

### **How to Reference the Semantic Model**:
The **Semantic Model** is stored in a **Snowflake Stage** as a YAML file. You can reference this model in API calls to perform queries.

```python
# Python function to retrieve the Semantic Model path from Snowflake stage
def fetch_semantic_model():
    # Path to the YAML file stored in the Snowflake Stage
    model_path = f"@SKICAR_SKICAR_SCHEMA_SETUP/SKICAR_Semantic_Model.yaml"
    return model_path

### **Using the Verified Query Repository**

The **Verified Query Repository** stores pre-approved SQL queries, ensuring that only validated and efficient queries are executed. This ensures optimized performance and superior query accuracy.

#### **Key Points**:
- **Stores Pre-approved Queries**: Manages and reuses efficient, validated queries.
- **Ensures Efficient Query Execution**: Only tested and optimized queries are used by **Cortex Analyst**.

#### **Example of a Verified Query**:
The following query compares the **average shipping time** for **Snowtires Automotive** against all other suppliers:

```sql
-- Example Verified Query to compare shipping times for 'Snowtires Automotive' vs. other suppliers
select DISTINCT(supplier_vendor_name), average_shipping_time 
from SKICAR.SKICAR_SCHEMA.SUPPLY_CHAIN 
where supplier_vendor_name = 'Snowtires Automotive'
union
select 'Other Suppliers' AS supplier_vendor_name, 
avg(average_shipping_time) 
from (select DISTINCT(supplier_vendor_name), average_shipping_time 
      from SKICAR.SKICAR_SCHEMA.SUPPLY_CHAIN 
      where supplier_vendor_name <> 'Snowtires Automotive');

## **Cortex Analyst API Integration**

The **Cortex Analyst API** connects Snowflake with natural language queries, translating them using the **Semantic Model**, and returning results.

### **Key Steps**:
1. **Send Natural Language Query**: The user asks a question via Slack (or another interface).
2. **API Call**: The query is sent to the **Cortex Analyst API**, which generates the corresponding SQL and returns the result.

### **Endpoint for the Cortex Analyst API**:
```python
# Endpoint for Cortex Analyst REST API
ANALYST_ENDPOINT = f"https://demo72.snowflakecomputing.com/api/v2/cortex/analyst/message"
```

### **Function to Call the Cortex Analyst API:**
```python
def query_cortex_analyst(question, model_path, pat_token):
    headers = {
        'Authorization': f'Bearer {pat_token}',  # Use Programmatic Access Token (PAT)
        'Content-Type': 'application/json'
    }
    
    # Prepare the request payload with the semantic model and query
    data = {
        "semantic_model": model_path,  # Path to your model in Snowflake stage
        "question": question
    }
    
    # Send POST request to the API
    response = requests.post(ANALYST_ENDPOINT, headers=headers, json=data)
    
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Error querying Cortex Analyst API: {response.status_code}, {response.text}")
```

## **Cortex Analyst API Integration via Slack**

This section demonstrates how the **Cortex Analyst API** is integrated with **Slack**, enabling users to query data directly through the chat interface. Users can submit questions via the **/askcortex** command, and the system processes these queries using the **Cortex Analyst API**.

This integration allows for real-time querying, providing immediate insights and seamless interaction with data through Slack.

### **What You Saw**:
- **Real-time Querying**: You saw how the **Cortex Analyst API** processes natural language queries from Slack and generates corresponding SQL queries to fetch the relevant data.
- **Instant Insights**: The system returned immediate insights, answering queries such as "What's the average shipping time from Snowtires Automotive?" directly in Slack.
  
### **Demo Flow Summary**:
1. **User Query**: The user asks a question via the **/askcortex** command in Slack.
2. **Cortex Analyst API**: The query is sent to the **Cortex Analyst API**, which generates the appropriate SQL based on the **Semantic Model**.
3. **SQL Execution**: The SQL is executed on the relevant Snowflake data sources, and the result is returned to the user.
4. **Real-time Insights**: The user receives instant data insights, all through a conversational interface in Slack.

Below is a screenshot showing the **Cortex Analyst REST API** in action, processing a query directly from Slack using the **/askcortex** command.


In [ ]:
# Define image in a stage and read the file
image=session.file.get_stream('@SKICAR.SKICAR_SCHEMA.SETUP/Slack.jpg' , decompress=False).read() 

# Display the image
st.image(image, width=1200)